In [1]:
## add ignore warnings for now, will remove and debug once full algorithm is complete
import warnings
warnings.filterwarnings("ignore")

## import packages/libraries
from time import perf_counter, clock_gettime_ns, CLOCK_REALTIME
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
from itertools import product
import sys
import sqlite3

## append filepath to allow files to be called from within project folder
sys.path.append('/home/gerard/Desktop/capstone_project/patoms')
sys.path.append('/home/gerard/Desktop/capstone_project')

## call locally created functions
from snapshot_2d_pattern_v6 import patoms2d
from snapshot_3d_pattern_v6 import patoms3d
from pattern_2d_compare_v5 import pattern_compare_2d
from pattern_3d_compare_v4 import pattern_compare_3d
from get_2d_table_names_v0 import table_names_2d
from updating_ref_table_v0 import update_ref_table
from patom_to_table_v0 import patom_to_table_func

In [2]:
## connect to 2d database
con2d = sqlite3.connect("ref2d_v4.db")
cur2d = con2d.cursor()
cur2d.execute('create table pat_2d_000000(norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, quad, quad_cnt, patom_ind, frame_ind_arr, patom_time, xc_d, yc_d, x_d, y_d);')
con2d.commit()

In [3]:
## read 2d ref database into memory and convert to ref patoms for comparison
working_ref_patoms = []

con2dref = sqlite3.connect("ref2d_v4.db")
con2dref.deserialize(f.read())
cur2dref = con2dref.cursor()
ref_names = [name for (name,) in cur2dref.execute("select name from sqlite_master where type='table';").fetchall()]
for i in ref_names:
    table = cur2dref.execute(f"select * from {i};").fetchall()
    table_array = np.array(table)
    working_ref_patoms.append(table_array)

ref_patoms_array = np.vstack(working_ref_patoms[:50])
ref_indices = np.unique(ref_patoms_array[:,6:8],axis=0)


In [ ]:
val = 0
while val <= 1:
    rand_array = np.random.random((1, 720, 1280))
    y_len = rand_array.shape[1]
    x_len = rand_array.shape[2]
    # ingest data frame by frame
    for frame in range(rand_array.shape[0]):
        #################################################################################
        ####################### FIRST TASK: FIND PATTERNS IN FRAME ######################
        #################################################################################
        frame_patoms = patoms2d(x_len, y_len, rand_array[frame,:,:], frame)
        # output: [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, quad, quad_cnt, patom_ind, frame_ind_arr, patom_time]
        num_patoms = len(frame_patoms)
        val += 1
        # ###########################################################################
        # ########## SECOND TASK: COMPARE NEW PATOMS AGAINST REF PATOMS #############
        # ###########################################################################
        atime = perf_counter()
        with Pool(processes=4) as pool:
            items = [(frame_patoms[i], ref_patoms_array) for i in range(num_patoms)]
            comp_results = pool.starmap(pattern_compare_2d, items) #e.g. ['px','py','pxc','pyc','pq','pqlen','ppind','pfind','ptime','rpind','rfind','xc_d','yc_d','x_d','y_d','similar']
            print("Time to compare 2D patterns with multiprocessing (secs):", (perf_counter()-atime))
            ## loop through the output of the comparison function
            for ix, i in enumerate(comp_results):
                # get max table number from 2d database
                next_table_num = int([table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()].pop(-1)[-6:]) + 1 
                next_table_num = str(next_table_num).zfill(6)
                # get existing table names
                existing_tables = [names for (names,) in cur2d.execute("select name from sqlite_master where type='table';").fetchall()]

                columns = np.array([0,1,2,3,4,5,6,7,8,11,12,13,14], dtype=np.intp)
                rows = np.array(range(np.unique(i[:,:1],axis=0).shape[0]), dtype=np.intp)
                patom = i[np.ix_(rows,columns)]

                similar_patoms = i[i[:,15] == 1]
                if similar_patoms.shape[0] > 0:
                    ref_patom_indices = similar_patoms[:,9:11] 
                    ref_table_name_indices = [xs for x in np.argwhere(ref_indices[np.isin(ref_indices, ref_patom_indices)]).tolist() for xs in x]
                    if ref_table_name_indices: # why are there empty lists??? there are empty lsts as there are ref indices that do not have any match to new similar patoms
                        ref_tables_numbers = [ref_names[i][-6:] for i in ref_table_name_indices]
                    for i in ref_tables_numbers:
                        cur2d.executemany(f"INSERT INTO pat_2d_{i}(norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, quad, quad_cnt, patom_ind, frame_ind_arr, patom_time, 
                                            xc_d, yc_d, x_d, y_d) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", patom)
                
                non_similar_patoms = i[i[:,4] != 1]
                if non_similar_patoms.shape[0] > 0:
                    cur2d.execute(f"CREATE TABLE pat_2d_{next_table_num}(norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, quad, quad_cnt, patom_ind, frame_ind_arr, 
                                    patom_time, xc_d, yc_d, x_d, y_d);")
                    cur2d.executemany(f"INSERT INTO pat_2d_{next_table_num}(norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, quad, quad_cnt, patom_ind, frame_ind_arr, patom_time, 
                                        xc_d, yc_d, x_d, y_d) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", patom)    
                
                con2d.commit()

                # check if table exists, if so, add patom, if not create table and add patom
                ref_table_num = ref_names[i[1]][-6:] # check this is correct
                new_patom = frame_patoms[i[0]] # check this is correct
                ptttime = perf_counter()
                patom_to_table = patom_to_table_func(new_patom)
                print('time to convert patom to table:',perf_counter()-ptttime)
                # add patom to relevant pattern table get name of ref table to get data table name
                if f"pat_2d_{ref_table_num}" in [names for (names,) in cur2d.execute("select name from sqlite_master where type='table';").fetchall()]:
                    cur2d.executemany(f"INSERT INTO pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                    con2d.commit()
                    # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
                else:
                    cur2d.execute(f"CREATE TABLE pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
                    cur2d.executemany(f"INSERT INTO pat_2d_{ref_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                    con2d.commit()
            # else:
            #     for i in remaining_comp_indexes:
            #         mttime = perf_counter()
            #         next_table_num = int([table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()].pop(-1)[-6:]) + 1 
            #         print('time to get max table:',perf_counter()-mttime)
            #         next_table_num = str(next_table_num).zfill(6)
            #         new_patom = frame_patoms[i] # checthis is correct
            #         ptttime = perf_counter()
            #         patom_to_table = patom_to_table_func(new_patom)
            #         print('time to convert patom to table:',perf_counter()-ptttime)
            #         # if patom is not matched then create new pattern & ref tables, get max table name and then create next new one
            #         cur2d.execute(f"CREATE TABLE pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
            #         cur2d.executemany(f"INSERT INTO pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
            #         con2d.commit()


Time to get 2D patterns with multiprocessing (secs): 0.4798233839974273
Time to compare 2D patterns with multiprocessing (secs): 5.965298764000181
(100, 13)
(89, 13)
(301, 13)
(26, 13)
(50, 13)
(254, 13)
(19, 13)
(15, 13)
(181, 13)
(15, 13)
(69, 13)
(27, 13)
(237, 13)
(213, 13)
(18, 13)
(31, 13)
(55, 13)
(68, 13)
(139, 13)
(36, 13)
(32, 13)
(9, 13)
(15, 13)
(75, 13)
(131, 13)
(99, 13)
(26, 13)
(87, 13)
(93, 13)
(316, 13)
(49, 13)
(33, 13)
(180, 13)
(144, 13)
(63, 13)
(238, 13)
(78, 13)
(159, 13)
(230, 13)
(62, 13)
(39, 13)
(51, 13)
(112, 13)
(15, 13)
(98, 13)
(21, 13)
(69, 13)
(57, 13)
(143, 13)
(240, 13)
(174, 13)
(13, 13)
(36, 13)
(52, 13)
(15, 13)
(120, 13)
(12, 13)
(61, 13)
(27, 13)
(12, 13)
(100, 13)
(15, 13)
(13, 13)
(92, 13)
(71, 13)
(33, 13)
(30, 13)
(85, 13)
(88, 13)
(55, 13)
(64, 13)
(21, 13)
(18, 13)
(55, 13)
(126, 13)
(12, 13)
(32, 13)
(251, 13)
(142, 13)
(22, 13)
(10, 13)
(18, 13)
(82, 13)
(106, 13)
(33, 13)
(54, 13)
(136, 13)
(175, 13)
(104, 13)
(64, 13)
(89, 13)
(113, 13

In [5]:
# ref_indices = np.unique(ref_patoms_array[:,6:8],axis=0)
# ref_patom_indices = similar_patoms[:,2:4]
# ref_table_name = ref_names[np.where(np.all(ref_indices==ref_patom_indices,axis=1))[0][0]]
# print(ref_table_name)
# frame_patoms_indices = np.unique(frame_patoms[78][:,6:8], axis=0)
# new_patoms_indices = similar_patoms[:,:2]
# patom = frame_patoms[np.where(np.all(frame_patoms_indices==new_patoms_indices,axis=1))[0][0]]
# print(patom)

In [6]:
# test_table = cur2dref.execute(f"select * from {ref_names[0]};").fetchall()
# test_table